In [1]:
!pip install --no-cache-dir -U git+https://github.com/codelion/optillm.git

  Cloning https://github.com/codelion/optillm.git to /private/var/folders/8z/jnnncfnj7_lfxym0262z4p180000gn/T/pip-req-build-iyrpe6t6
  Running command git clone --filter=blob:none --quiet https://github.com/codelion/optillm.git /private/var/folders/8z/jnnncfnj7_lfxym0262z4p180000gn/T/pip-req-build-iyrpe6t6
  Resolved https://github.com/codelion/optillm.git to commit 7429aec15f671749ddab560c3e4beff769fd361e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optillm.cot_decoding import cot_decode
import torch


def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")


model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)



/Users/ben/code/argilla-cookbook/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from datasets import load_dataset

ds = load_dataset("AI-MO/NuminaMath-CoT")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
messages = [
    {
        "role": "user",
        "content": "In a dance class of 20 students, 20% enrolled in contemporary dance, 25% of the remaining enrolled in jazz dance, and the rest enrolled in hip-hop dance. What percentage of the entire students enrolled in hip-hop dance?",
    }
]

# Generate the response using CoT decoding
result, confidence = cot_decode(
    model, tokenizer, messages, aggregate_paths=True, max_new_tokens=512
)
device = get_device()
model = model.to(device)
# print(f"Using device: {get_device()}")

print(f"CoT Decoding:\n {result}")
# print(f"Confidence: {confidence}")

input_ids = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"
)
input_ids = input_ids.to(device)
input_length = input_ids.shape[1]
output_ids = model.generate(
    input_ids,
    max_new_tokens=512,  # Adjust this value based on how long you want the response to be
    do_sample=False,  # This ensures greedy decoding
    num_beams=1,
)
output_ids = output_ids.cpu()
# Decode only the newly generated tokens
response = tokenizer.decode(output_ids[0][input_length:], skip_special_tokens=True)

print(f"Greedy Decoding:\n {response}")

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.
/Users/ben/code/argilla-cookbook/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/ben/code/argilla-cookbook/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should s

CoT Decoding:
 1. **Determine the number of students enrolled in contemporary dance:**
   \[
   20 \text{ students} \times 20\% = 20 \times 0.20 = 4 \text{ students}
   \]

2. **Calculate the number of remaining students after subtracting those who are enrolled in contemporary dance:**
   \[
   20 \text{ students} - 4 \text{ students} = 16 \text{ students}
   \]

3. **Determine the number of students enrolled in jazz dance:**
   \[
   16 \text{ students} \times 25\% = 16 \times 0.25 = 4 \text{ students}
   \]

4. **Calculate the number of remaining students after subtracting those who are enrolled in jazz dance:**
   \[
   16 \text{ students} - 4 \text{ students} = 12 \text{ students}
   \]

5. **Determine the number of students enrolled in hip-hop dance:**
   \[
   \text{Total students} - \text{Students enrolled in contemporary dance} - \text{Students enrolled in jazz dance} = 20 \text{ students} - 4 \text{ students} - 4 \text{ students} = 12 \text{ students}
   \]

6. **Calculate the

In [4]:
result

'1. **Determine the number of students enrolled in contemporary dance:**\n   \\[\n   20 \\text{ students} \\times 20\\% = 20 \\times 0.20 = 4 \\text{ students}\n   \\]\n\n2. **Calculate the number of remaining students after subtracting those who are enrolled in contemporary dance:**\n   \\[\n   20 \\text{ students} - 4 \\text{ students} = 16 \\text{ students}\n   \\]\n\n3. **Determine the number of students enrolled in jazz dance:**\n   \\[\n   16 \\text{ students} \\times 25\\% = 16 \\times 0.25 = 4 \\text{ students}\n   \\]\n\n4. **Calculate the number of remaining students after subtracting those who are enrolled in jazz dance:**\n   \\[\n   16 \\text{ students} - 4 \\text{ students} = 12 \\text{ students}\n   \\]\n\n5. **Determine the number of students enrolled in hip-hop dance:**\n   \\[\n   \\text{Total students} - \\text{Students enrolled in contemporary dance} - \\text{Students enrolled in jazz dance} = 20 \\text{ students} - 4 \\text{ students} - 4 \\text{ students} = 12 \\t